In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost
from sklearn.cross_validation import train_test_split

C:\Users\satyam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\satyam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
train_data=pd.read_csv("train_data_processed.csv")
test_data=pd.read_csv("test_data_processed.csv")

In [12]:
def pre_process(data,train):
    if train==True:
        data = data[data["Open"] != 0]
        data = data[data["Sales"] >0]
        data.loc[data.Promo2SinceYear == 0, 'PromoOpen'] = 0
    data=data.drop(columns=["Date","Id","CompetitionOpenSinceYear","Promo2SinceWeek","Promo2SinceYear","Month","Day","Year",
            "monthStr","WeekOfYear","Unnamed: 0","PromoInterval"])
    return data

In [15]:
train_data=pre_process(train_data,True)

Exception ignored in: <function DMatrix.__del__ at 0x000001E1EB3629D8>
Traceback (most recent call last):
  File "C:\Users\satyam\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [16]:
test_data=pre_process(test_data,False)

In [17]:
train_data.to_csv("train_data_processed1.csv")
test_data.to_csv("test_data_processed1.csv")

In [34]:
features=list(train_data.columns)

In [35]:
features.remove("Sales")

In [36]:
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.3,
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1,
          "seed": 1301
          }
num_boost_round = 300
X_train, X_valid = train_test_split(train_data, test_size=0.012, random_state=10)
y_train = np.log1p(X_train.Sales)
y_valid = np.log1p(X_valid.Sales)
dtrain = xgboost.DMatrix(X_train[features], y_train)
dvalid = xgboost.DMatrix(X_valid[features], y_valid)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgboost.train(params, dtrain, num_boost_round, evals=watchlist, \
  early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)


[0]	train-rmse:5.78963	eval-rmse:5.79328	train-rmspe:0.996914	eval-rmspe:0.996923
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.
[1]	train-rmse:4.05767	eval-rmse:4.06116	train-rmspe:0.981878	eval-rmspe:0.981929
[2]	train-rmse:2.84266	eval-rmse:2.84548	train-rmspe:0.939725	eval-rmspe:0.939868
[3]	train-rmse:1.99289	eval-rmse:1.9954	train-rmspe:0.860235	eval-rmspe:0.860557
[4]	train-rmse:1.40008	eval-rmse:1.40207	train-rmspe:0.747996	eval-rmspe:0.74843
[5]	train-rmse:0.98438	eval-rmse:0.986263	train-rmspe:0.620163	eval-rmspe:0.620808
[6]	train-rmse:0.696872	eval-rmse:0.698733	train-rmspe:0.494129	eval-rmspe:0.494995
[7]	train-rmse:0.494686	eval-rmse:0.495868	train-rmspe:0.382886	eval-rmspe:0.383554
[8]	train-rmse:0.355234	eval-rmse:0.356224	train-rmspe:0.292064	eval-rmspe:0.292752
[9]	train-rmse:0.259951	eval-rmse:0.260945	train-rmspe:0.222654	eval-rmspe:0.223492
[10]	train-rmse:0.196715	ev

[96]	train-rmse:0.056968	eval-rmse:0.061763	train-rmspe:0.056582	eval-rmspe:0.061583
[97]	train-rmse:0.056926	eval-rmse:0.061758	train-rmspe:0.056541	eval-rmspe:0.061576
[98]	train-rmse:0.056807	eval-rmse:0.061665	train-rmspe:0.056427	eval-rmspe:0.061489
[99]	train-rmse:0.056707	eval-rmse:0.061638	train-rmspe:0.056331	eval-rmspe:0.061458
[100]	train-rmse:0.056593	eval-rmse:0.061612	train-rmspe:0.056228	eval-rmspe:0.061432
[101]	train-rmse:0.056506	eval-rmse:0.061569	train-rmspe:0.056142	eval-rmspe:0.061392
[102]	train-rmse:0.056449	eval-rmse:0.061571	train-rmspe:0.056083	eval-rmspe:0.061403
[103]	train-rmse:0.056324	eval-rmse:0.061428	train-rmspe:0.055961	eval-rmspe:0.061269
[104]	train-rmse:0.056235	eval-rmse:0.061375	train-rmspe:0.055871	eval-rmspe:0.061211
[105]	train-rmse:0.056199	eval-rmse:0.061359	train-rmspe:0.055836	eval-rmspe:0.061198
[106]	train-rmse:0.056091	eval-rmse:0.061286	train-rmspe:0.055728	eval-rmspe:0.061125
[107]	train-rmse:0.056015	eval-rmse:0.061272	train-rmspe:0

[192]	train-rmse:0.050629	eval-rmse:0.058849	train-rmspe:0.05039	eval-rmspe:0.058746
[193]	train-rmse:0.050585	eval-rmse:0.058841	train-rmspe:0.050345	eval-rmspe:0.058736
[194]	train-rmse:0.0505	eval-rmse:0.058813	train-rmspe:0.050263	eval-rmspe:0.058709
[195]	train-rmse:0.05045	eval-rmse:0.058787	train-rmspe:0.050213	eval-rmspe:0.058681
[196]	train-rmse:0.050422	eval-rmse:0.058764	train-rmspe:0.050185	eval-rmspe:0.058654
[197]	train-rmse:0.050379	eval-rmse:0.05876	train-rmspe:0.050146	eval-rmspe:0.058652
[198]	train-rmse:0.050325	eval-rmse:0.05873	train-rmspe:0.050092	eval-rmspe:0.058626
[199]	train-rmse:0.05027	eval-rmse:0.058723	train-rmspe:0.050043	eval-rmspe:0.058617
[200]	train-rmse:0.050218	eval-rmse:0.058706	train-rmspe:0.04999	eval-rmspe:0.058602
[201]	train-rmse:0.050171	eval-rmse:0.058681	train-rmspe:0.049944	eval-rmspe:0.058579
[202]	train-rmse:0.050121	eval-rmse:0.058668	train-rmspe:0.049894	eval-rmspe:0.058563
[203]	train-rmse:0.05008	eval-rmse:0.058663	train-rmspe:0.0498

[288]	train-rmse:0.04685	eval-rmse:0.057687	train-rmspe:0.046677	eval-rmspe:0.057647
[289]	train-rmse:0.046816	eval-rmse:0.057694	train-rmspe:0.046644	eval-rmspe:0.057653
[290]	train-rmse:0.046769	eval-rmse:0.057692	train-rmspe:0.046595	eval-rmspe:0.057651
[291]	train-rmse:0.046731	eval-rmse:0.057696	train-rmspe:0.04656	eval-rmspe:0.057655
[292]	train-rmse:0.046704	eval-rmse:0.057697	train-rmspe:0.046533	eval-rmspe:0.057655
[293]	train-rmse:0.046699	eval-rmse:0.057694	train-rmspe:0.046528	eval-rmspe:0.057649
[294]	train-rmse:0.046651	eval-rmse:0.057694	train-rmspe:0.046487	eval-rmspe:0.057653
[295]	train-rmse:0.046634	eval-rmse:0.057705	train-rmspe:0.046469	eval-rmspe:0.057665
[296]	train-rmse:0.046603	eval-rmse:0.057697	train-rmspe:0.046438	eval-rmspe:0.057658
[297]	train-rmse:0.046585	eval-rmse:0.057685	train-rmspe:0.046421	eval-rmspe:0.057645
[298]	train-rmse:0.046569	eval-rmse:0.057686	train-rmspe:0.046404	eval-rmspe:0.057645
[299]	train-rmse:0.046533	eval-rmse:0.057683	train-rmspe

In [37]:
dtest = xgboost.DMatrix(test_data[features])

In [38]:
test_probs = gbm.predict(dtest)

In [41]:
test_data.columns

Index(['Store', 'DayOfWeek', 'Customers', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'Promo2', 'CompetitionOpenMonths',
       'PromoOpen', 'IsPromoMonth'],
      dtype='object')

In [42]:
test_data=pd.read_csv("test_data_processed.csv")
result = pd.DataFrame({"Id": test_data["Id"], 'Sales': np.expm1(test_probs)})
result.to_csv("Submission13.csv")

In [43]:
random_model=RandomForestRegressor()
random_model.fit(train_data[features].values,train_data['Sales'].values.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [44]:
predictions2=random_model.predict(test_data[features].values)

In [45]:
result = pd.DataFrame({"Id": test_data["Id"], 'Sales': predictions2})
result.to_csv("Submission14.csv")